In [1]:
def method_holm(pvalues, alpha=0.05):
    """Применяет метод Холма для проверки значимости изменений.
    
    pvalues - List[float] - список pvalue.
    alpha - float, уровень значимости.
    return - np.array, массив из нулей и единиц, 0 - эффекта нет, 1 - эффект есть.
    """
    m = len(pvalues)
    array_alpha = np.arange(m, 0, -1)
    array_alpha = alpha / array_alpha
    sorted_pvalue_indexes = np.argsort(pvalues)
    res = np.zeros(m)
    for idx, pvalue_index in enumerate(sorted_pvalue_indexes):
        pvalue = pvalues[pvalue_index]
        alpha_ = array_alpha[idx]
        if pvalue < alpha_:
            res[pvalue_index] = 1
        else:
            break
    res = res.astype(int)
    return res

In [2]:
# Проверим, что при 12 эксперимента ошибки контролируются на заданных уровнях, а при 13 экспериментах нет.

for count_exp in [12, 13]:
    errors_aa = []
    errors_ab = []
    sample_size = int(total_size / (int(count_exp) * 2))
    for _ in tqdm(range(10000)):
        list_ab_values = [
            np.random.normal(mean_, std_, (2, sample_size))
            for _ in range(count_exp)
        ]
        # синтетический А/А тест
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        aa_with_effect = method_holm(pvalues, alpha)
        errors_aa.append(np.sum(aa_with_effect) > 0)

        # Синтетический А/Б тест.
        # Достаточно проверить случай, когда эффект есть лишь в одном из экспериментов,
        # так как при наличии эффектов в большем кол-ве экспериментов ошибок II рода станет меньше.
        # Добавим эффект в первый эксперимент (не важно в какой добавлять, так как данные случайные)
        list_ab_values[0][1] *= 1 + effect
        pvalues = [stats.ttest_ind(a, b).pvalue for a, b in list_ab_values]
        ab_with_effect = method_holm(pvalues, alpha)
        if np.sum(ab_with_effect) == 0:
            # если эффектов не найдено, то это ошибка
            errors_ab.append(True)
        else:
            # если эффектов найден где его нет, то это ошибка
            errors_ab.append(np.min(pvalues) != pvalues[0])

    estimated_first_type_error = np.mean(errors_aa)
    estimated_second_type_error = np.mean(errors_ab)
    ci_first = estimate_ci_bernoulli(estimated_first_type_error, len(errors_aa))
    ci_second = estimate_ci_bernoulli(estimated_second_type_error, len(errors_ab))
    print(f'count_exp = {count_exp}')
    print(f'sample_size = {sample_size}')
    print(f'оценка вероятности ошибки I рода = {estimated_first_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_first[0]:0.4f}, {ci_first[1]:0.4f}]')
    print(f'оценка вероятности ошибки II рода = {estimated_second_type_error:0.4f}')
    print(f'  доверительный интервал = [{ci_second[0]:0.4f}, {ci_second[1]:0.4f}]')

NameError: name 'total_size' is not defined